In [ ]:
#| hide
from OPUS.core import *

# OPUS

> A Python library for reading and parsing [NOAA OPUS reports.](https://www.ngs.noaa.gov/OPUS/index.jsp)

This function extracts most of the information from a NOAA OPUS report, or Extended report. NOAA's [OPUS ( Online Positioning User Service)](https://www.ngs.noaa.gov/OPUS/view.jsp) provides free access to high-accuracy National Spatial Reference System (NRCS) coordinates.  To use OPUS
upload a RINEX data file collected with a survey grade GNSS to OPUS and it will compute the precise position for you. The OPUS report will 
will be emailed to you within a few minutes.  Once you receive the OPUS report, paste the contents to a file.  Once you have saved the report to
a file, you can use decode_OPUS() to easily extract any of the data to your Python program.  decode_OPUS() can read directly from an OPUS file
or a string containing the contents of an OPUS file.

## Install

```sh
pip install OPUS
```

## Dev Install

When working on the `decode_OPUS()` function, use the `Dev Install`.
``` sh
pip -e install OPUS
```

## Simple use

Using decode_OPUS() is easy. Simply install and then import OPUS.

In [ ]:
import OPUS

In [ ]:
from OPUS import OPUS_test_data
OPUS_test_data.opus_txt_string

"\nFILE: SFKW1210.24O OP1716177177447\n\n 1008   NOTE: You provided a zero or negative antenna height.\n 1008   If ARP HGT = 0.0, OPUS solves for the position of your selected antenna's reference point (ARP).\n 1008   If ARP HGT < 0.0, OPUS solves for a location inside or above the antenna\n 1008\n                              NGS OPUS SOLUTION REPORT\n                              ========================\n\nAll computed coordinate accuracies are listed as peak-to-peak values.\nFor additional information: https://www.ngs.noaa.gov/OPUS/about.jsp#accuracy\n\n      USER: xxxxx@yyyy.com                           DATE: May 20, 2024\nRINEX FILE: sfkw121p.24o                             TIME: 03:53:37 UTC\n\n\n  SOFTWARE: page5  2008.25 master253.pl 160321      START: 2024/04/30  15:51:00\n EPHEMERIS: igs23122.eph [precise]                   STOP: 2024/04/30  22:51:00\n  NAV FILE: brdc1210.24n                         OBS USED: 14342 / 14604   :  98%\n  ANT NAME: TRMR10-2        NONE         

To use decode_OPUS(), simply call it with the filename of an OPUS file and it will return an object containing 
most of the data extracted from the OPUS file.

In [ ]:
opus = OPUS.decode_OPUS( opus_file = '../test_data/OPUS-california.txt' )

Print the NAD83 latitude, longitude, ellipsoidal and orthometric heights and the difference in heights.

In [ ]:
print(
  "                                                      Elevations (m)   Elevation Dif (m)\n",
  "   OPUS File    Latitude (deg)   Longitude (deg) Ellipsodal   NAVD88 Ellipsoidal-NAVD88\n",
  opus.rinex_file,
  opus.nad83.lat, 
  opus.nad83.lon, 
  opus.nad83.el_hgt,
  opus.nad83.navd88_hgt,
  opus.nad83.navd88_hgt - opus.nad83.el_hgt
)

                                                      Elevations (m)   Elevation Dif (m)
    OPUS File    Latitude (deg)   Longitude (deg) Ellipsodal   NAVD88 Ellipsoidal-NAVD88
 sfkw121p.24o 36.35005248888889 -118.76384246944444 1094.647 1122.958 28.31100000000015


Print the WGS84/ITRF latitude, longitude, ellipsoidal, orthometric and the difference in heights.

In [ ]:
print(
  "  WGS84 / ITRF                                 Ellipsodal Elevations (m) Elevation Dif (m)\n",
  "   OPUS File    Latitude (deg)     Longitude (deg)  WGS84   NAD83  Ellipsoidal-NAVD88\n",
  opus.rinex_file,
  opus.itrf.lat, 
  opus.itrf.lon, 
  opus.itrf.el_hgt,
  opus.nad83.el_hgt,
  opus.nad83.el_hgt - opus.itrf.el_hgt
)

  WGS84 / ITRF                                 Ellipsodal Elevations (m) Elevation Dif (m)
    OPUS File    Latitude (deg)     Longitude (deg)  WGS84   NAD83  Ellipsoidal-NAVD88
 sfkw121p.24o 36.35005613888889 -118.76386053888889 1094.0 1094.647 0.6469999999999345


In [ ]:
optxt = OPUS.decode_OPUS( txt = OPUS.opus_txt_string )

In [ ]:
optxt.nad83.lat, optxt.nad83.lon, optxt.nad83.navd88_hgt

(36.35005248888889, -118.76384246944444, 1122.958)

### Basic OPUS data.

Returned OPUS data is returned in a class object which contains basic OPUS data and three sub-classes.
In the cell below, we will dump and examine the elements contained in the basic OPUS class.

In [ ]:
print("Key           Value")
for k in optxt.__dict__:
  print(f"{k:13} {optxt.__dict__[k]}")

Key           Value
nad83         <OPUS.decode_OPUS.decode_OPUS.<locals>.POINT object>
itrf          <OPUS.decode_OPUS.decode_OPUS.<locals>.POINT object>
state_plane   <OPUS.decode_OPUS.decode_OPUS.<locals>.POINT object>
strings       <OPUS.decode_OPUS.decode_OPUS.<locals>.STRINGS object>
title         NGS OPUS SOLUTION REPORT
user          xxxxx@yyyy.com
run_date      2024-05-20 00:00:00
rinex_file    sfkw121p.24o
run_datetime  2024-05-20 03:53:37
software      page5  2008.25 master253.pl 160321
start         2024-04-30 15:51:00
ephermis      precise
stop          2024-04-30 22:51:00
ant_name      TRMR10-2        NONE
arp_height    0.0
lat_rms       0.022
lon_rms       0.012
el_rms        0.012
utm_zone      11
northing      4024219.772
easting       341727.091
us_grid       11SLA4172724220


### nad83 class data.

OPUS computes coordinates in the [North American Datum of 1983](https://en.wikipedia.org/wiki/North_American_Datum#North_American_Datum_of_1983).  
We will examine the `nad83` subclass.  The `nad83` subclass contains the ECEF x,y,z coordinates as well as
the floating point values for latitude `lat`, longitude `lon`, and NAD83 ellipsoidal elevation `el_hgt` and the `navd88_hgt`. 
The [NAVD88](https://geodesy.noaa.gov/datums/vertical/north-american-vertical-datum-1988.shtml)
data is only available in the US.

In [ ]:
print("Key           Value\n------------------------------")
for k in optxt.nad83.__dict__:
  print(f"{k:13} {optxt.nad83.__dict__[k]}")

Key           Value
------------------------------
ref_frame     NAD_83(2011)(EPOCH:2010.0000)
x             -2475273.328
y             -4509243.734
z             3760194.885
lat           36.35005248888889
lon           -118.76384246944444
el_hgt        1094.647
navd88_hgt    1122.958


### ITRF class data.

In addition to the `nad83`, OPUS also reports the position in the
[International Terresterial Reference Frame (ITRF)](https://en.wikipedia.org/wiki/International_Terrestrial_Reference_System_and_Frame) for both
[ECEF](https://en.wikipedia.org/wiki/Earth-centered,_Earth-fixed_coordinate_system) and geodetic ellipsodal coordinates.

In [ ]:
print("Key           Value\n------------------------------")
for k in optxt.itrf.__dict__:
  print(f"{k:13} {optxt.itrf.__dict__[k]}")

Key           Value
------------------------------
ref_frame     ITRF2014 (EPOCH:2024.3301)
x             -2475274.384
y             -4509242.286
z             3760194.828
lat           36.35005613888889
lon           -118.76386053888889
el_hgt        1094.0


### State Plane class data.

State Plane data will be present when the OPUS point is in the United States.

In [ ]:
print("Key           Value\n------------------------------")
for k in optxt.state_plane.__dict__:
  print(f"{k:13} {optxt.state_plane.__dict__[k]}")

Key           Value
------------------------------
id            0404 CA 4
northing      612842.492
easting       2021197.302


### OPUS reported Dates and times

 OPUS reported dates and times are converted to [python datetime](https://docs.python.org/3/library/datetime.html)
 data types to make calculations easier.  Below we will compute the occupation time from the decoded Start and Stop times in the OPUS report.

In [ ]:
occupation_time = (optxt.stop - optxt.start)
occupation_time.seconds / 3600
print(
  f"Start time: { optxt.start}\n"
  f" Stop time: { optxt.stop}\n"
  f"The position was occupied for {occupation_time.seconds/3600:5.2f} hours."
)

Start time: 2024-04-30 15:51:00
 Stop time: 2024-04-30 22:51:00
The position was occupied for  7.00 hours.
